In [1]:
# Install some packages
if (!require("ggpubr")) install.packages("ggpubr")
if (!require("ggplot2")) install.packages("ggplot2")
if (!require("tiff")) install.packages("tiff")
if (!require("grid")) install.packages("grid")
if (!require("gridExtra")) install.packages("gridExtra")
if (!require("ggseg")) install.packages("ggseg")
if (!require("R.matlab")) install.packages("R.matlab")

Loading required package: ggpubr

Loading required package: ggplot2

Loading required package: tiff

Loading required package: grid

Loading required package: gridExtra

Loading required package: ggseg

Loading required package: R.matlab

R.matlab v3.7.0 (2022-08-25 21:52:34 UTC) successfully loaded. See ?R.matlab for help.


Attaching package: ‘R.matlab’


The following objects are masked from ‘package:base’:

    getOption, isOpen




In [2]:
data=read.table('/dagher/dagher11/filip/UPF/data/extracted_data_updated_correctedencoding_nutrients.csv', header=TRUE, na.strings = "", sep=',', quote = "\"")
icddate=read.table('/dagher/dagher11/filip/UPF/data/icd10date.csv', header=TRUE, na.strings = "", sep=',', quote = "\"")
data=merge(data,icddate)

In [3]:
exclusions_neuro=c('^G','^E0','^E10','^E2','^E3','^A8','^C70','^C71','^C72','^I6')
# G - nervous system; F - Mental and behavioural; E0 - thyroid; E11-14 - diabetes, E2-3 endocrine glands; 
# A8 - viral infections of the CNS; C70-72 malignant neoplasms of brain, meninges and spinal cord; 
# I6 - cerebrovscular

icd_dates_cols=grep('icd10_41280', colnames(data))
index=1
data$neurological_disorder = NA

for (i in as.numeric(grep('icd10_41270', colnames(data)))) {
    for (d in 1:length(exclusions_neuro)) {
        data$neurological_disorder[grepl(exclusions_neuro[d], data[[i]]) & 
                                   (data[icd_dates_cols[index]]<data$`date_of_attending_assessment_centre_53.2.0` | 
                                   data[icd_dates_cols[index]]<data$`date_of_attending_assessment_centre_53.0.0`)] = 1
    }
    index=index+1
}

data$excluded <- NA
data$excluded[data$neurological_disorder ==1] <- 1

data$included<-car::recode(data$excluded, "1='excluded'; else='included'")
nrow(data)
#Select subjects if included 
data_excluded <- subset(data, included=="included")

ukbb_all=data_excluded

ukbb_all$WHR = ukbb_all$waist_circumference_48.0.0/ukbb_all$hip_circumference_49.0.0
ukbb_all$agesq=ukbb_all$age_when_attended_assessment_centre_21003.2.0^2

nrow(ukbb_all)

[1] 502293

[1] 459349

In [4]:
write.csv(ukbb_all, '/dagher/dagher11/filip/UPF/data/dataset_excluded.csv', quote=T, row.names=F)